In [1]:
import pandas as pd
import numpy as np
import gzip
import json
import shutil
from pathlib import Path

The aim is to read the data from a CSV file after wrangling it, and then read it with csv.DictReader and append it to the JSON file we created.

# processing data in csv file

The overall purpose of this wrangle() function is to read a CSV file, perform various data cleaning and transformation steps, and return the resulting DataFrame with the modifications applied.

In [2]:
def wrangle(file_path):
    df = pd.read_csv(file_path,low_memory=False,encoding='latin-1')
    
    start_index = 1  # Starting index for the new column names
    column_range = range(1, 96)  # Range of columns to be renamed

    for i, col_index in enumerate(column_range):
        new_col_name = f"Attr_{start_index + i}"
        df.rename(columns={df.columns[col_index]: new_col_name}, inplace=True)
    
    # insert column 'company_id'
    df.insert(0, 'id', range(1, len(df) + 1))

    # rename 'class' column to 'bankrupt'
    df = df.rename(columns={'Flag':'bankrupt'})
    
    df["bankrupt"]=[True if v==1 else False for v in df["bankrupt"]]
    
    # remove '?' values 
    df = df.replace({'?': 'NaN'})
    
#     df= df.astype({c: "float64" for c in df.columns[1:65]})
#     df['bankrupt'] = df['bankrupt'].astype(bool)
    
    return df

In [3]:
df = wrangle(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy.csv")
print(df.shape)
df.head()

(6819, 97)


,id,bankrupt,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,1,True,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,2,True,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,3,True,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,4,True,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,5,True,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [4]:
df["bankrupt"].value_counts()

False    6599
True      220
Name: bankrupt, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 97 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        6819 non-null   int64  
 1   bankrupt  6819 non-null   bool   
 2   Attr_1    6819 non-null   float64
 3   Attr_2    6819 non-null   float64
 4   Attr_3    6819 non-null   float64
 5   Attr_4    6819 non-null   float64
 6   Attr_5    6819 non-null   float64
 7   Attr_6    6819 non-null   float64
 8   Attr_7    6819 non-null   float64
 9   Attr_8    6819 non-null   float64
 10  Attr_9    6819 non-null   float64
 11  Attr_10   6819 non-null   float64
 12  Attr_11   6819 non-null   float64
 13  Attr_12   6819 non-null   float64
 14  Attr_13   6819 non-null   float64
 15  Attr_14   6819 non-null   float64
 16  Attr_15   6819 non-null   float64
 17  Attr_16   6819 non-null   float64
 18  Attr_17   6819 non-null   float64
 19  Attr_18   6819 non-null   float64
 20  Attr_19   6819 non-null   floa

### Save changes we made on csv file by wrangle function to new csv file `Taiwan_bankruptcy_modified.csv`

In [6]:
df.to_csv(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.csv", index=False)

In [7]:
df_test=pd.read_csv(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.csv")
df_test.head()

,id,bankrupt,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,1,True,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,2,True,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,3,True,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,4,True,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,5,True,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [8]:
df_test["bankrupt"].value_counts()

False    6599
True      220
Name: bankrupt, dtype: int64

###### After procession  the CSV file,we extracts random rows, sorts them based on the first column, saves them to a new CSV file, and removes the extracted rows from the original file.

In [9]:
import csv
import random

input_file = r'E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.csv'
output_file = r'E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.csv'

# Step 1: Read the CSV file
with open(input_file, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

    header_row = rows[0]
    data_rows = rows[1:]

    num_data_rows = len(data_rows)
    num_random_rows = 682  # Define the number of random rows you want to extract
    random_indices = random.sample(range(num_data_rows), num_random_rows)

    random_rows = [data_rows[index] for index in random_indices]

    # Step 2: Sort the extracted rows based on the first column
    sorted_rows = sorted(random_rows, key=lambda row: float(row[0]))

    # Step 3: Save the sorted extracted rows in a new CSV file with the header
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(header_row)  # Write the header row
        writer.writerows(sorted_rows)

    # Step 4: Remove the extracted rows from the original CSV file
    remaining_rows = [row for index, row in enumerate(data_rows) if index not in random_indices]

    with open(input_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header_row)  # Write the header row
        writer.writerows(remaining_rows)

In [10]:
original_csv = pd.read_csv(r'E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.csv')
print(original_csv.shape)
original_csv.head()

(6137, 97)


,id,bankrupt,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,1,True,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,2,True,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,3,True,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,4,True,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,5,True,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [11]:
original_csv['bankrupt'].value_counts()

False    5943
True      194
Name: bankrupt, dtype: int64

In [12]:
# remove bankrupt column from Taiwan_bankruptcy_modified_test.csv
import csv
filename = r'E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.csv'
column_index = 1

with open(filename, 'r', newline='') as file:
    reader = csv.reader(file)
    rows = [row[:column_index] + row[column_index+1:] for row in reader]

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

In [13]:
extracted_csv = pd.read_csv(r'E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.csv')
print(extracted_csv.shape)
extracted_csv.head()

(682, 96)


,id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,10,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,...,0.804195,0.005798,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
1,13,0.491152,0.551570,0.543391,0.608131,0.608138,0.999045,0.797429,0.809344,0.303435,...,0.803260,0.008232,0.623578,0.608125,0.840702,0.281113,0.027480,0.567146,1,0.025282
2,40,0.486228,0.548027,0.544140,0.598445,0.598445,0.998953,0.797404,0.809343,0.303584,...,0.801372,0.003425,0.623089,0.598440,0.840386,0.277247,0.028873,0.568501,1,0.047960
3,41,0.507824,0.571631,0.549226,0.625326,0.625326,0.999136,0.797542,0.809413,0.303442,...,0.815731,0.007173,0.623729,0.625327,0.841574,0.279236,0.026908,0.565659,1,0.031485
4,71,0.544825,0.588149,0.589218,0.600636,0.600636,0.999039,0.797469,0.809371,0.303517,...,0.827985,0.004125,0.623411,0.600633,0.842319,0.278315,0.026871,0.565512,1,0.036871


### Create JSON file 

In [14]:
Taiwan_bankruptcy_modified = {
     "schema": {
               "fields":[{'name': 'company_id', 'type': 'integer'},
                         {'name': 'bankrupt', 'type': 'boolean'},
                         {'name': 'Attr_1', 'type': 'number'},
                         {'name': 'Attr_2', 'type': 'number'},
                         {'name': 'Attr_3', 'type': 'number'},
                         {'name': 'Attr_4', 'type': 'number'},
                         {'name': 'Attr_5', 'type': 'number'},
                         {'name': 'Attr_6', 'type': 'number'},
                         {'name': 'Attr_7', 'type': 'number'},
                         {'name': 'Attr_8', 'type': 'number'},
                         {'name': 'Attr_9', 'type': 'number'},
                         {'name': 'Attr_10', 'type': 'number'},
                         {'name': 'Attr_11', 'type': 'number'},
                         {'name': 'Attr_12', 'type': 'number'},
                         {'name': 'Attr_13', 'type': 'number'},
                         {'name': 'Attr_14', 'type': 'number'},
                         {'name': 'Attr_15', 'type': 'number'},
                         {'name': 'Attr_16', 'type': 'number'},
                         {'name': 'Attr_17', 'type': 'number'},
                         {'name': 'Attr_18', 'type': 'number'},
                         {'name': 'Attr_19', 'type': 'number'},
                         {'name': 'Attr_20', 'type': 'number'},
                         {'name': 'Attr_21', 'type': 'number'},
                         {'name': 'Attr_22', 'type': 'number'},
                         {'name': 'Attr_23', 'type': 'number'},
                         {'name': 'Attr_24', 'type': 'number'},
                         {'name': 'Attr_25', 'type': 'number'},
                         {'name': 'Attr_26', 'type': 'number'},
                         {'name': 'Attr_27', 'type': 'number'},
                         {'name': 'Attr_28', 'type': 'number'},
                         {'name': 'Attr_29', 'type': 'number'},
                         {'name': 'Attr_30', 'type': 'number'},
                         {'name': 'Attr_31', 'type': 'number'},
                         {'name': 'Attr_32', 'type': 'number'},
                         {'name': 'Attr_33', 'type': 'number'},
                         {'name': 'Attr_34', 'type': 'number'},
                         {'name': 'Attr_35', 'type': 'number'},
                         {'name': 'Attr_36', 'type': 'number'},
                         {'name': 'Attr_37', 'type': 'number'},
                         {'name': 'Attr_38', 'type': 'number'},
                         {'name': 'Attr_39', 'type': 'number'},
                         {'name': 'Attr_40', 'type': 'number'},
                         {'name': 'Attr_41', 'type': 'number'},
                         {'name': 'Attr_42', 'type': 'number'},
                         {'name': 'Attr_43', 'type': 'number'},
                         {'name': 'Attr_44', 'type': 'number'},
                         {'name': 'Attr_45', 'type': 'number'},
                         {'name': 'Attr_46', 'type': 'number'},
                         {'name': 'Attr_47', 'type': 'number'},
                         {'name': 'Attr_48', 'type': 'number'},
                         {'name': 'Attr_49', 'type': 'number'},
                         {'name': 'Attr_50', 'type': 'number'},
                         {'name': 'Attr_51', 'type': 'number'},
                         {'name': 'Attr_52', 'type': 'number'},
                         {'name': 'Attr_53', 'type': 'number'},
                         {'name': 'Attr_54', 'type': 'number'},
                         {'name': 'Attr_55', 'type': 'number'},
                         {'name': 'Attr_56', 'type': 'number'},
                         {'name': 'Attr_57', 'type': 'number'},
                         {'name': 'Attr_58', 'type': 'number'},
                         {'name': 'Attr_59', 'type': 'number'},
                         {'name': 'Attr_60', 'type': 'number'},
                         {'name': 'Attr_61', 'type': 'number'},
                         {'name': 'Attr_62', 'type': 'number'},
                         {'name': 'Attr_63', 'type': 'number'},
                         {'name': 'Attr_64', 'type': 'number'},
                         {'name': 'Attr_65', 'type': 'number'},
                         {'name': 'Attr_66', 'type': 'number'},
                         {'name': 'Attr_67', 'type': 'number'},
                         {'name': 'Attr_68', 'type': 'number'},
                         {'name': 'Attr_69', 'type': 'number'},
                         {'name': 'Attr_70', 'type': 'number'},
                         {'name': 'Attr_71', 'type': 'number'},
                         {'name': 'Attr_72', 'type': 'number'},
                         {'name': 'Attr_73', 'type': 'number'},
                         {'name': 'Attr_74', 'type': 'number'},
                         {'name': 'Attr_75', 'type': 'number'},
                         {'name': 'Attr_76', 'type': 'number'},
                         {'name': 'Attr_77', 'type': 'number'},
                         {'name': 'Attr_78', 'type': 'number'},
                         {'name': 'Attr_79', 'type': 'number'},
                         {'name': 'Attr_80', 'type': 'number'},
                         {'name': 'Attr_81', 'type': 'number'},
                         {'name': 'Attr_82', 'type': 'number'},
                         {'name': 'Attr_83', 'type': 'number'},
                         {'name': 'Attr_84', 'type': 'number'},
                         {'name': 'Attr_85', 'type': 'integer'},
                         {'name': 'Attr_86', 'type': 'number'},
                         {'name': 'Attr_87', 'type': 'number'},
                         {'name': 'Attr_88', 'type': 'number'},
                         {'name': 'Attr_89', 'type': 'number'},
                         {'name': 'Attr_90', 'type': 'number'},
                         {'name': 'Attr_91', 'type': 'number'},
                         {'name': 'Attr_92', 'type': 'number'},
                         {'name': 'Attr_93', 'type': 'number'},
                         {'name': 'Attr_94', 'type': 'integer'},
                         {'name': 'Attr_95', 'type': 'number'}],
               "primary_key":["company_id"],
                "pandas_version":"0.20.0"
              },
    "observations": [],
    "metadata": {'title': 'Financial ratios and corporate governance indicators in bankruptcy prediction: A comprehensive study',
                 'authors': 'Deron Liang, Chia-Chi Lu, Chih-Fong Tsai, Guan-An Shih',
                 'journal': 'European Journal of Operational Research',
                 'publicationYear': 2016,
                 'articleLink': 'https://doi.org/10.1016/j.ejor.2016.01.012',
                 'datasetLink': 'https://archive.ics.uci.edu/ml/datasets/Taiwanese+Bankruptcy+Prediction'}
    }
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json","w") as file:
    json.dump(Taiwan_bankruptcy_modified,file,ensure_ascii=False,indent=4)

By the end of the loop, the `dict_data` list will contain dictionaries, with each dictionary representing a row of data from the CSV file.

# Note

The csv.`DictReader` function in Python's csv module reads values from a CSV file as strings by default. This behavior is intentional because CSV files are text-based and do not have explicit data types defined for the values. Therefore, all values in the CSV file are initially treated as strings.

So if we append the data from a CSV file as dictionaries, the data types of values will be treated as strings. To avoid this problem, there are two solutions: one is to fix the data types of values before appending them to the JSON file, and the other is to fix the data types of values after appending them to the JSON file.

##### In this Notebook, we will fix the data types of values after appending them to the JSON file.

In [15]:
dict_data_1=[]
import csv
        
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.csv", newline='') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
                
        dict_data_1.append(row)

In [16]:

json_file = Path(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json")
json_data = json.loads(json_file.read_text())

data = [index for index in dict_data_1]
json_data["observations"] = data

json_file.write_text(json.dumps(json_data))

18720448

In [17]:
def wrangle_1(filename):
    
    # Open compressed file, load into dictionary
    with open(filename,"r") as f:
        data = json.load(f)
    # Load dictionary into DataFrame, set index
    df=pd.DataFrame.from_dict(data["observations"])
    
    return df

In [31]:
df1 = wrangle_1(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json")
print(df1.shape)
df1.head()

(6137, 97)


,id,bankrupt,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,1,True,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,2,True,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,3,True,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,4,True,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,5,True,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [32]:
df1["bankrupt"].value_counts()

False    5943
True      194
Name: bankrupt, dtype: int64

In [20]:
# Step 1: Read the JSON file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json", 'r') as file:
    data = json.load(file)

# Step 2: Traverse and locate the value to change
list_item = data["observations"][0:]  
for dictionary in list_item:

    for key, value in dictionary.items():
        
        if value=='':
            dictionary[key] = None
            
        elif key=='id':
            dictionary[key] = int(value)
        
        
        elif key=='bankrupt':
            dictionary[key] = True if value == "True" else False
        
        
        
        elif key.startswith('Attr') and key not in ['Attr_85', 'Attr_94']:
            dictionary[key] = float(value)
            
        elif key  in ['Attr_85','Attr_94']:
            dictionary[key] = int(value)     
            
        else :
            pass


# Step 3: Write the modified object back to the JSON file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json", 'w') as file:
    json.dump(data, file, indent=4)

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6137 entries, 0 to 6136
Data columns (total 97 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6137 non-null   object
 1   bankrupt  6137 non-null   object
 2   Attr_1    6137 non-null   object
 3   Attr_2    6137 non-null   object
 4   Attr_3    6137 non-null   object
 5   Attr_4    6137 non-null   object
 6   Attr_5    6137 non-null   object
 7   Attr_6    6137 non-null   object
 8   Attr_7    6137 non-null   object
 9   Attr_8    6137 non-null   object
 10  Attr_9    6137 non-null   object
 11  Attr_10   6137 non-null   object
 12  Attr_11   6137 non-null   object
 13  Attr_12   6137 non-null   object
 14  Attr_13   6137 non-null   object
 15  Attr_14   6137 non-null   object
 16  Attr_15   6137 non-null   object
 17  Attr_16   6137 non-null   object
 18  Attr_17   6137 non-null   object
 19  Attr_18   6137 non-null   object
 20  Attr_19   6137 non-null   object
 21  Attr_20   6137

In [22]:
# gzip compress file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json",'rb') as f_input:
    with gzip.open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified.json.gz",'wb') as f_output:
        shutil.copyfileobj(f_input,f_output)


In [23]:
Taiwan_bankruptcy_modified_test = {
     "schema": {
               "fields":[{'name': 'company_id', 'type': 'integer'},
                         {'name': 'Attr_1', 'type': 'number'},
                         {'name': 'Attr_2', 'type': 'number'},
                         {'name': 'Attr_3', 'type': 'number'},
                         {'name': 'Attr_4', 'type': 'number'},
                         {'name': 'Attr_5', 'type': 'number'},
                         {'name': 'Attr_6', 'type': 'number'},
                         {'name': 'Attr_7', 'type': 'number'},
                         {'name': 'Attr_8', 'type': 'number'},
                         {'name': 'Attr_9', 'type': 'number'},
                         {'name': 'Attr_10', 'type': 'number'},
                         {'name': 'Attr_11', 'type': 'number'},
                         {'name': 'Attr_12', 'type': 'number'},
                         {'name': 'Attr_13', 'type': 'number'},
                         {'name': 'Attr_14', 'type': 'number'},
                         {'name': 'Attr_15', 'type': 'number'},
                         {'name': 'Attr_16', 'type': 'number'},
                         {'name': 'Attr_17', 'type': 'number'},
                         {'name': 'Attr_18', 'type': 'number'},
                         {'name': 'Attr_19', 'type': 'number'},
                         {'name': 'Attr_20', 'type': 'number'},
                         {'name': 'Attr_21', 'type': 'number'},
                         {'name': 'Attr_22', 'type': 'number'},
                         {'name': 'Attr_23', 'type': 'number'},
                         {'name': 'Attr_24', 'type': 'number'},
                         {'name': 'Attr_25', 'type': 'number'},
                         {'name': 'Attr_26', 'type': 'number'},
                         {'name': 'Attr_27', 'type': 'number'},
                         {'name': 'Attr_28', 'type': 'number'},
                         {'name': 'Attr_29', 'type': 'number'},
                         {'name': 'Attr_30', 'type': 'number'},
                         {'name': 'Attr_31', 'type': 'number'},
                         {'name': 'Attr_32', 'type': 'number'},
                         {'name': 'Attr_33', 'type': 'number'},
                         {'name': 'Attr_34', 'type': 'number'},
                         {'name': 'Attr_35', 'type': 'number'},
                         {'name': 'Attr_36', 'type': 'number'},
                         {'name': 'Attr_37', 'type': 'number'},
                         {'name': 'Attr_38', 'type': 'number'},
                         {'name': 'Attr_39', 'type': 'number'},
                         {'name': 'Attr_40', 'type': 'number'},
                         {'name': 'Attr_41', 'type': 'number'},
                         {'name': 'Attr_42', 'type': 'number'},
                         {'name': 'Attr_43', 'type': 'number'},
                         {'name': 'Attr_44', 'type': 'number'},
                         {'name': 'Attr_45', 'type': 'number'},
                         {'name': 'Attr_46', 'type': 'number'},
                         {'name': 'Attr_47', 'type': 'number'},
                         {'name': 'Attr_48', 'type': 'number'},
                         {'name': 'Attr_49', 'type': 'number'},
                         {'name': 'Attr_50', 'type': 'number'},
                         {'name': 'Attr_51', 'type': 'number'},
                         {'name': 'Attr_52', 'type': 'number'},
                         {'name': 'Attr_53', 'type': 'number'},
                         {'name': 'Attr_54', 'type': 'number'},
                         {'name': 'Attr_55', 'type': 'number'},
                         {'name': 'Attr_56', 'type': 'number'},
                         {'name': 'Attr_57', 'type': 'number'},
                         {'name': 'Attr_58', 'type': 'number'},
                         {'name': 'Attr_59', 'type': 'number'},
                         {'name': 'Attr_60', 'type': 'number'},
                         {'name': 'Attr_61', 'type': 'number'},
                         {'name': 'Attr_62', 'type': 'number'},
                         {'name': 'Attr_63', 'type': 'number'},
                         {'name': 'Attr_64', 'type': 'number'},
                         {'name': 'Attr_65', 'type': 'number'},
                         {'name': 'Attr_66', 'type': 'number'},
                         {'name': 'Attr_67', 'type': 'number'},
                         {'name': 'Attr_68', 'type': 'number'},
                         {'name': 'Attr_69', 'type': 'number'},
                         {'name': 'Attr_70', 'type': 'number'},
                         {'name': 'Attr_71', 'type': 'number'},
                         {'name': 'Attr_72', 'type': 'number'},
                         {'name': 'Attr_73', 'type': 'number'},
                         {'name': 'Attr_74', 'type': 'number'},
                         {'name': 'Attr_75', 'type': 'number'},
                         {'name': 'Attr_76', 'type': 'number'},
                         {'name': 'Attr_77', 'type': 'number'},
                         {'name': 'Attr_78', 'type': 'number'},
                         {'name': 'Attr_79', 'type': 'number'},
                         {'name': 'Attr_80', 'type': 'number'},
                         {'name': 'Attr_81', 'type': 'number'},
                         {'name': 'Attr_82', 'type': 'number'},
                         {'name': 'Attr_83', 'type': 'number'},
                         {'name': 'Attr_84', 'type': 'number'},
                         {'name': 'Attr_85', 'type': 'integer'},
                         {'name': 'Attr_86', 'type': 'number'},
                         {'name': 'Attr_87', 'type': 'number'},
                         {'name': 'Attr_88', 'type': 'number'},
                         {'name': 'Attr_89', 'type': 'number'},
                         {'name': 'Attr_90', 'type': 'number'},
                         {'name': 'Attr_91', 'type': 'number'},
                         {'name': 'Attr_92', 'type': 'number'},
                         {'name': 'Attr_93', 'type': 'number'},
                         {'name': 'Attr_94', 'type': 'integer'},
                         {'name': 'Attr_95', 'type': 'number'}],
               "primary_key":["company_id"],
                "pandas_version":"0.20.0"
              },
    "observations": [],
    "metadata": {'title': 'Financial ratios and corporate governance indicators in bankruptcy prediction: A comprehensive study',
                 'authors': 'Deron Liang, Chia-Chi Lu, Chih-Fong Tsai, Guan-An Shih',
                 'journal': 'European Journal of Operational Research',
                 'publicationYear': 2016,
                 'articleLink': 'https://doi.org/10.1016/j.ejor.2016.01.012',
                 'datasetLink': 'https://archive.ics.uci.edu/ml/datasets/Taiwanese+Bankruptcy+Prediction'}
    }
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json","w") as file:
    json.dump(Taiwan_bankruptcy_modified_test,file,ensure_ascii=False,indent=4)

In [24]:
dict_data_2=[]
import csv
        
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.csv", newline='') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
               
        dict_data_2.append(row)

In [25]:
json_file = Path(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json")
json_data = json.loads(json_file.read_text())

data = [index for index in dict_data_2]
json_data["observations"] = data

json_file.write_text(json.dumps(json_data))

2069750

In [26]:
def wrangle_2(filename):
    
    # Open compressed file, load into dictionary
    with open(filename,"r") as f:
        data = json.load(f)
    # Load dictionary into DataFrame, set index
    df=pd.DataFrame.from_dict(data["observations"])
    
    return df

In [27]:
df2 = wrangle_2(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json")
print(df2.shape)
df2.head()

(682, 96)


,id,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_86,Attr_87,Attr_88,Attr_89,Attr_90,Attr_91,Attr_92,Attr_93,Attr_94,Attr_95
0,10,0.495685662750451,0.550915830789359,0.542962685368596,0.599208694273483,0.599208694273483,0.999000632043326,0.797403570816415,0.809320385467476,0.303482953821731,...,0.804194656500343,0.005797642207721,0.623865043621207,0.599205074060972,0.840688177069564,0.279387518743906,0.0272430149466587,0.566667622359207,1,0.0308008650901007
1,13,0.491151952420416,0.551569995638901,0.543390973820868,0.608130702373917,0.608137909165598,0.999044886630839,0.797428887121513,0.809344341779496,0.303434855738379,...,0.803260160615522,0.008232490111815,0.62357757367742,0.608125444361497,0.840702415877267,0.281113383864046,0.0274797246787734,0.567146326659175,1,0.0252817202305257
2,40,0.486228245502852,0.548026602703881,0.544140478612345,0.598444774355353,0.598444774355353,0.998953142133487,0.797404400859205,0.809343300200712,0.30358374226775,...,0.801371569744629,0.0034249823452305,0.623089158340436,0.598440276500219,0.840385757478808,0.277247090635208,0.0288727658138465,0.568500549655657,1,0.0479599415621359
3,41,0.507824306537318,0.571631051024858,0.549226403983083,0.625326107323542,0.625326107323542,0.999135706750384,0.7975416346005,0.8094133463739,0.303441623307896,...,0.815731085142529,0.007173154537515,0.623729409498007,0.625326634759901,0.841573616502083,0.279235950325587,0.026907690710701,0.565658865268012,1,0.0314853592017782
4,71,0.544825232779213,0.588148713475796,0.589217838214037,0.600635639026218,0.600635639026218,0.999038647080641,0.797468867515902,0.809370902038474,0.303517275067138,...,0.827984672530329,0.0041247924464135,0.623411192820107,0.600633492612391,0.842318562340046,0.278315149342696,0.026870881562317,0.565511873396239,1,0.0368709257372074


In [28]:
# Step 1: Read the JSON file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json", 'r') as file:
    data = json.load(file)

# Step 2: Traverse and locate the value to change
list_item = data["observations"][0:]  
for dictionary in list_item:

    for key, value in dictionary.items():
        
        if value=='':
            dictionary[key] = None
            
        elif key=='id':
            dictionary[key] = int(value)
        
        elif key.startswith('Attr') and key not in ['Attr_85', 'Attr_94']:
            dictionary[key] = float(value)
            
        elif key  in ['Attr_85','Attr_94']:
            dictionary[key] = int(value)     
            
        else :
            pass


# Step 3: Write the modified object back to the JSON file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json", 'w') as file:
    json.dump(data, file, indent=4)

In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 96 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       682 non-null    object
 1   Attr_1   682 non-null    object
 2   Attr_2   682 non-null    object
 3   Attr_3   682 non-null    object
 4   Attr_4   682 non-null    object
 5   Attr_5   682 non-null    object
 6   Attr_6   682 non-null    object
 7   Attr_7   682 non-null    object
 8   Attr_8   682 non-null    object
 9   Attr_9   682 non-null    object
 10  Attr_10  682 non-null    object
 11  Attr_11  682 non-null    object
 12  Attr_12  682 non-null    object
 13  Attr_13  682 non-null    object
 14  Attr_14  682 non-null    object
 15  Attr_15  682 non-null    object
 16  Attr_16  682 non-null    object
 17  Attr_17  682 non-null    object
 18  Attr_18  682 non-null    object
 19  Attr_19  682 non-null    object
 20  Attr_20  682 non-null    object
 21  Attr_21  682 non-null    object
 22  At

In [30]:
# gzip compress file
with open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json",'rb') as f_input:
    with gzip.open(r"E:\WorldQuant\project_5\data\Taiwan_bankruptcy_modified_test.json.gz",'wb') as f_output:
        shutil.copyfileobj(f_input,f_output)